In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

def mult(x):
    return 10*x


def max(x,y):
    if x > y:
        out = x
    else:
        out = y
    return out


def MyReLU(x):
    if x > 0:
        output = x
    else:
        output = torch.zeros_like(x, requires_grad=True)
    return output


a  = torch.tensor(5., requires_grad=True)
b = 5*a

#c = b
c = mult(b)
#c = max(a,b)
#c = MyReLU(b**2)
#c = MyReLU(-b**2)  ## The last one won't work for gradient compute


c.backward()
print(a.grad)

tensor(50.)


In [3]:
# https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html

class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        
        if input > 0:
            output = input
        else:
            output = torch.zeros_like(input, requires_grad=True)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [4]:
relu = MyReLU().apply

a  = torch.tensor(5., requires_grad=True)
b = 5*a
c = relu(-b**2)

c.backward()
print(a.grad)

tensor(-0.)


tensor([ 0.8000,  0.4000, -1.2000], grad_fn=<quantBackward>)
